# Session 4 — Transformations: Joins, Grouping, Pivoting

Combine tables and create analytics-friendly shapes. Emulate pivots with conditional aggregation.


## Environment Setup

In [ ]:
import sys, sqlite3, pandas as pd, numpy as np, matplotlib.pyplot as plt
print(sys.version)
import seaborn as sns
sns.set_theme()
from pathlib import Path
DB_PATH = Path('course.db')
conn = sqlite3.connect(DB_PATH)
conn.execute('PRAGMA foreign_keys=ON;')
print('SQLite ready at', DB_PATH.resolve())

In [ ]:
def run_sql(q, params=None):
    params = params or {}
    df = pd.read_sql_query(q, conn, params=params)
    display(df)
    return df

## 1. Joins Refresher & Examples

In [ ]:
run_sql('''
SELECT o.order_id, c.name AS customer, o.order_date, s.name AS shipper
FROM orders o
JOIN customers c ON o.customer_id=c.customer_id
LEFT JOIN shippers s ON o.shipper_id=s.shipper_id
ORDER BY o.order_id;
''')

## 2. Aggregations and HAVING

In [ ]:
run_sql('''
SELECT c.city, SUM(oi.quantity*p.price) AS revenue
FROM orders o
JOIN order_items oi ON o.order_id=oi.order_id
JOIN products p ON oi.product_id=p.product_id
JOIN customers c ON o.customer_id=c.customer_id
GROUP BY c.city
HAVING revenue > 100
ORDER BY revenue DESC;
''')

## 3. Pivot-Style Summary (Conditional Aggregation)

In [ ]:
run_sql('''
SELECT c.country,
       SUM(CASE WHEN p.product='Keyboard' THEN oi.quantity ELSE 0 END) AS keyboards,
       SUM(CASE WHEN p.product='Mouse' THEN oi.quantity ELSE 0 END) AS mice,
       SUM(CASE WHEN p.product='Monitor' THEN oi.quantity ELSE 0 END) AS monitors,
       SUM(CASE WHEN p.product='USB-C Cable' THEN oi.quantity ELSE 0 END) AS cables
FROM orders o
JOIN order_items oi ON o.order_id=oi.order_id
JOIN products p ON oi.product_id=p.product_id
JOIN customers c ON o.customer_id=c.customer_id
GROUP BY c.country
ORDER BY c.country;
''')

## 4. Visualize Revenue by City

In [ ]:
df_rev = pd.read_sql_query('''
SELECT c.city AS city, SUM(oi.quantity*p.price) AS revenue
FROM orders o
JOIN order_items oi ON o.order_id=oi.order_id
JOIN products p ON oi.product_id=p.product_id
JOIN customers c ON o.customer_id=c.customer_id
GROUP BY c.city
ORDER BY revenue DESC;
''', conn)
sns.barplot(data=df_rev, x="city", y="revenue")
plt.title("Revenue by City"); plt.xlabel("City"); plt.ylabel("Revenue"); plt.show()